In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def query_skyscanner(date):


    url = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"

    payload = { "query": {
            "market": "UK",
            "locale": "en-GB",
            "currency": "EUR",
            "queryLegs": [
                {
                    "originPlace": { "queryPlace": { "iata": "LIS" } },
                    "destinationPlace": { "queryPlace": { "iata": "LTN" } },
                    "fixedDate": {
                        "year": date.year,
                        "month": date.month,
                        "day": date.day,
                    },
                }
            ],
        }
    }

    headers = {
        "x-api-key": "prtl6749387986743898559646983194"
    }

    response = requests.post(url, json=payload, headers=headers)

    output = response.json()

    if 'content' not in output:
        return None

    results = output['content']['results']

    if 'quotes' not in results:
        return None

    quotes = results['quotes']
    quotes_df = pd.json_normalize(quotes)
    quotes_df.columns = [col.split(".")[-1] for col in quotes_df.columns]

    def flatten_dict(dd, separator='_', prefix=''):
        return {f"{prefix}{separator}{k}" if prefix else k: v
                for kk, vv in dd.items()
                for k, v in (flatten_dict(vv, separator, kk).items() if isinstance(vv, dict) else {kk: vv}.items())}

    flattened_results = [flatten_dict(result) for result in quotes.values()]
    quote_df = pd.DataFrame.from_records(flattened_results)
    return quote_df

start_date = datetime(2023, 5, 20)
end_date = datetime(2023, 5, 22)

current_date = start_date

while current_date <= end_date:
    result = query_skyscanner(current_date)
    if result is not None:
        print(f"Results for {current_date.strftime('%Y-%m-%d')}:")
        print(result)
        print("\n")
    else:
        print(f"No results for {current_date.strftime('%Y-%m-%d')}\n")

    current_date += timedelta(days=1)


Results for 2023-05-20:
  minPrice_amount     minPrice_unit            minPrice_updateStatus  \
0              92  PRICE_UNIT_WHOLE  PRICE_UPDATE_STATUS_UNSPECIFIED   

   isDirect outboundLeg_originPlaceId outboundLeg_destinationPlaceId  \
0      True                  95565055                       95565053   

   outboundLeg_departureDateTime_year  outboundLeg_departureDateTime_month  \
0                                2023                                    5   

   outboundLeg_departureDateTime_day  outboundLeg_departureDateTime_hour  ...  \
0                                 20                                   0  ...   

   inboundLeg_originPlaceId  inboundLeg_destinationPlaceId  \
0                                                            

  inboundLeg_departureDateTime_year inboundLeg_departureDateTime_month  \
0                                 0                                  0   

  inboundLeg_departureDateTime_day inboundLeg_departureDateTime_hour  \
0                   

In [2]:
results_20 = pd.DataFrame({
    "minPrice_amount": [68],
    "minPrice_unit": ["PRICE_UNIT_WHOLE"],
    "minPrice_updateStatus": ["PRICE_UPDATE_STATUS_UNSPECIFIED"],
    "isDirect": [True],
    "outboundLeg_originPlaceId": [95565055],
    "outboundLeg_destinationPlaceId": [95565053],
    "outboundLeg_departureDateTime_year": [2023],
    "outboundLeg_departureDateTime_month": [5],
    "outboundLeg_departureDateTime_day": [20],
    "outboundLeg_departureDateTime_hour": [0],
})

results_21 = pd.DataFrame({
    "minPrice_amount": [123, 144],
    "minPrice_unit": ["PRICE_UNIT_WHOLE", "PRICE_UNIT_WHOLE"],
    "minPrice_updateStatus": ["PRICE_UPDATE_STATUS_UNSPECIFIED", "PRICE_UPDATE_STATUS_UNSPECIFIED"],
    "isDirect": [False, True],
    "outboundLeg_originPlaceId": [95565055, 95565055],
    "outboundLeg_destinationPlaceId": [95565053, 95565053],
    "outboundLeg_departureDateTime_year": [2023, 2023],
    "outboundLeg_departureDateTime_month": [5, 5],
    "outboundLeg_departureDateTime_day": [21, 21],
    "outboundLeg_departureDateTime_hour": [0, 0],
})

results_22 = pd.DataFrame({
    "minPrice_amount": [93],
    "minPrice_unit": ["PRICE_UNIT_WHOLE"],
    "minPrice_updateStatus": ["PRICE_UPDATE_STATUS_UNSPECIFIED"],
    "isDirect": [True],
    "outboundLeg_originPlaceId": [95565055],
    "outboundLeg_destinationPlaceId": [95565053],
    "outboundLeg_departureDateTime_year": [2023],
    "outboundLeg_departureDateTime_month": [5],
    "outboundLeg_departureDateTime_day": [22],
    "outboundLeg_departureDateTime_hour": [0],
})

result = pd.concat([results_20, results_21, results_22], axis=0, ignore_index=True)

print(result)

   minPrice_amount     minPrice_unit            minPrice_updateStatus  \
0               68  PRICE_UNIT_WHOLE  PRICE_UPDATE_STATUS_UNSPECIFIED   
1              123  PRICE_UNIT_WHOLE  PRICE_UPDATE_STATUS_UNSPECIFIED   
2              144  PRICE_UNIT_WHOLE  PRICE_UPDATE_STATUS_UNSPECIFIED   
3               93  PRICE_UNIT_WHOLE  PRICE_UPDATE_STATUS_UNSPECIFIED   

   isDirect  outboundLeg_originPlaceId  outboundLeg_destinationPlaceId  \
0      True                   95565055                        95565053   
1     False                   95565055                        95565053   
2      True                   95565055                        95565053   
3      True                   95565055                        95565053   

   outboundLeg_departureDateTime_year  outboundLeg_departureDateTime_month  \
0                                2023                                    5   
1                                2023                                    5   
2                            

In [3]:
column_rename_map = {
    "minPrice_amount": "Price",
    "minPrice_unit": "Currency Unit",
    "minPrice_updateStatus": "Update Status",
    "isDirect": "Direct Flight",
    "outboundLeg_originPlaceId": "Origin Place ID",
    "outboundLeg_destinationPlaceId": "Destination Place ID",
    "outboundLeg_departureDateTime_year": "Departure Year",
    "outboundLeg_departureDateTime_month": "Departure Month",
    "outboundLeg_departureDateTime_day": "Departure Day",
    "outboundLeg_departureDateTime_hour": "Departure Hour",
    
}

result.rename(columns=column_rename_map, inplace=True)


selected_columns = [
    "Departure Year",
    "Departure Month",
    "Departure Day",
    "Departure Hour",
    "Price",
    "Currency Unit",
    "Direct Flight",
    "Origin Place ID",
    "Destination Place ID",
]

display_result = result[selected_columns]
display_result


,Departure Year,Departure Month,Departure Day,Departure Hour,Price,Currency Unit,Direct Flight,Origin Place ID,Destination Place ID
0,2023,5,20,0,68,PRICE_UNIT_WHOLE,True,95565055,95565053
1,2023,5,21,0,123,PRICE_UNIT_WHOLE,False,95565055,95565053
2,2023,5,21,0,144,PRICE_UNIT_WHOLE,True,95565055,95565053
3,2023,5,22,0,93,PRICE_UNIT_WHOLE,True,95565055,95565053


In [14]:
def query_skyscanner(date):
    pass

def luke_function(origin, destination, start_date, end_date):
    results = pd.DataFrame()

    current_date = start_date

    while current_date <= end_date:
        result = query_skyscanner(current_date)
        if result is not None:
            results = pd.concat([results, result], ignore_index=True)
        current_date += timedelta(days=1)

    return results

